In [ ]:
%matplotlib qt
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from mne.io import read_raw_brainvision, read_raw_fif

from scipy.signal import find_peaks
import numpy as np
from mne.channels import read_dig_captrak, make_standard_montage
from mne.preprocessing import ICA
from mne_icalabel import label_components
import customtkinter as ctk
from mne.coreg import Coregistration
from mne import (open_report, events_from_annotations, Epochs, write_trans,
                setup_source_space, make_bem_model, make_bem_solution, make_forward_solution, compute_covariance,
                extract_label_time_course, read_labels_from_annot)
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from scipy.signal import find_peaks
import warnings
from mne.preprocessing import ICA, EOGRegression, create_eog_epochs, create_ecg_epochs, compute_proj_ecg, compute_proj_eog, EOGRegression

In [ ]:
%%latex
##### for preprocessing()
## ask philip where is PO7, PO8, O1 and O2 and adjust it here
## put these cahnnels as misc and see what we can do with it
## kernel crash when running ctk
## ssp for ecg (pulse)
## ssp for vertical eye
## regression for horizontal eye
## .load_data()
## add teas
## GSR, X, Y, Y drop
## rename y-axis in pulse plot maybe *1e9 and time shows
## mne.preprocessing.annotate_break
## interpolate bads (mne.preprocessing.find_bad_channels_lof)



##### for run_analysis()
## in gpias dont do baselining
## double check with repo codes
## ask gpias triggers code
## documenatiaion
## 

In [ ]:
import mne
import numpy as np
raw = mne.io.read_raw("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest/raw_prep.fif")

events, event_ids = mne.events_from_annotations(raw)
if "New Segment/" in event_ids: 
    events = events[events[:, 2] != event_ids["New Segment/"]]
    event_ids.pop("New Segment/")

In [ ]:
raw.plot()

In [ ]:
events, event_ids = mne.events_from_annotations(raw)
if "New Segment/" in event_ids: 
    events = events[events[:, 2] != event_ids["New Segment/"]]
    event_ids.pop("New Segment/")

        
thr = 30 * 250 # 30 seconds
events = events[np.where(np.diff(events[:, 0], prepend=-np.inf) < thr)[0]]
unique_ids, counts = np.unique(events[:, -1], return_counts=True)
keep_ids = unique_ids[counts > 1]
events = events[np.isin(events[:, -1], keep_ids)]

In [ ]:
## be cautios that eyes open interval is one less than eyes closed (maybe in the function I should add dictionary of events)

events_ec = events[:, 0][events[:, 2] == 6]  ## assume 6 is eyes closed
events_eo = events[:, 0][events[:, 2] == 4]  ## assume 4 is eyes open
num_intervals = min(len(events_ec), len(events_eo))

if len(events_ec) != len(events_eo):
    raise ValueError("Number of eyes open and eyes close events dons not match.")

raws_ec, raws_eo = [], []
for ec_s, eo_s in zip(events_ec, events_eo):
    tmin, tmax = ec_s / 250, eo_s / 250
    raws_ec.append(raw.copy().crop(tmin=tmin, tmax=tmax))

for ec_o, ec_s in zip(events_eo[:-1], events_ec[1:]):
    tmin, tmax = ec_o / 250, ec_s / 250
    raws_eo.append(raw.copy().crop(tmin=tmin, tmax=tmax))

epochs_ec, epochs_eo = [mne.make_fixed_length_epochs(
                                                    mne.concatenate_raws(raw_e),
                                                    duration=2
                                                    ) for raw_e in [raws_ec, raws_eo]]

In [ ]:
raw.info["projs"]

In [ ]:
events_eo[:-1]
        

In [ ]:
for event_idx, event in enumerate(events[::2, 0]):
    print(event_idx, event)
    if not event == events[-1][0]:
        tmin, tmax = event / 250, events[event_idx + 1][0] / 250
        print(tmin, tmax)

In [ ]:
events, event_ids = mne.events_from_annotations(raw)
if "New Segment/" in event_ids: 
    events = events[events[:, 2] != event_ids["New Segment/"]]
    event_ids.pop("New Segment/")

        
thr = 30 * 250 # 30 seconds
events = events[np.where(np.diff(events[:, 0], prepend=-np.inf) < thr)[0]]
unique_ids, counts = np.unique(events[:, -1], return_counts=True)
keep_ids = unique_ids[counts > 1]
events = events[np.isin(events[:, -1], keep_ids)]

raws_eo, raws_ec = [], []
for event_idx, event in enumerate(events[::2, 0]):
    if not event == events[-1][0]:
        tmin, tmax = event / 250, events[event_idx + 1][0] / 250
        raws_eo.append(raw.copy().crop(tmin=tmin, tmax=tmax))
for event_idx, event in enumerate(events[1::2, 0]):
    if not event == events[-1][0]:
        tmin, tmax = event / 250, events[event_idx + 1][0] / 250
        raws_ec.append(raw.copy().crop(tmin=tmin, tmax=tmax))

epochs_eo, epochs_ec = [mne.make_fixed_length_epochs(
                                                mne.concatenate_raws(raw_e),
                                                duration=2
                                                ) for raw_e in [raws_eo, raws_ec]]

In [ ]:
raw.plot(events=events)

In [ ]:
events, event_ids = mne.events_from_annotations(raw)
if "New Segment/" in event_ids: 
    events = events[events[:, 2] != event_ids["New Segment/"]]
    event_ids.pop("New Segment/")


thr = 30 * 250 # 30 seconds
events = events[np.where(np.diff(events[:, 0], prepend=-np.inf) < thr)[0]]
unique_ids, counts = np.unique(events[:, -1], return_counts=True)
keep_ids = unique_ids[counts > 1]
events = events[np.isin(events[:, -1], keep_ids)]

In [ ]:
%matplotlib qt
from eeg_processing import preprocessing
preprocessing(subject_id="zdfy",
                subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="rest_v1",
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                eog_correct=True,
                pulse_correct=True,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                verbose="ERROR")

In [1]:
%matplotlib qt
from eeg_rs_processing import run_rs_analysis

run_rs_analysis(
        subject_id="zdfy",
        subjects_dir="/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
        visit=1,
        event_ids=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )


 17%|███████▌                                     

Loading preprocessed EEG data ...

Creating epochs...



 33%|███████████████                              

Loading MRI information of Freesurfer template subject ...

Computing forward solution ...



OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
 42%|██████████████████▊                          

Using ad hoc noise covariance for the recording ...

Computing the minimum-norm inverse solution ...



 58%|██████████████████████████▎                  

Apply inverse solution and extract label time courses...



TypeError: unhashable type: 'list'

In [2]:
import mne
epochs_ec = mne.read_epochs("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest_v1/epochs-ec-epo.fif")
epochs_eo = mne.read_epochs("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest_v1/epochs-eo-epo.fif")

In [ ]:
label_ts = {"eyes_close": [], "eyes_open": []}
for epochs, title in zip([epochs_ec, epochs_eo], list(label_ts.keys())):
    label_ts[title] = epochs

TypeError: unhashable type: 'list'

In [ ]:
label_ts.keys().

dict_keys(['eyes_close', 'eyes_open'])

In [ ]:
import mne
raw = mne.io.read_raw("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/rest/raw_prep.fif", preload=True)

In [ ]:
raw.plot()

In [ ]:
import mne
report = mne.open_report("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/reports/rest.h5")
report.save("/Users/payamsadeghishabestari/Desktop/check.html", overwrite=True)

Eyetracking

In [ ]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [ ]:
raw.annotations

In [ ]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [ ]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

In [ ]:
raw_eye.plot()

In [ ]:
raw_eye.plot()